In [20]:
!pip install mdtex2html

  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
Using cached Markdown-3.7-py3-none-any.whl (106 kB)


In [32]:
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 101.9 MB/s eta 0:00:00


In [38]:
!pip install markdown

In [2]:
import json
from pyprojroot import here
from datasets import Dataset
import mdtex2html
import matplotlib.pyplot as plt
from markdown import markdown

In [2]:
par_dir = here("struct_vs_unstruct/data/mistral_large_2407/original/non_self_synthesis/math/math-/math_eval")

In [3]:
ds = Dataset.load_from_disk(par_dir)
ds

Dataset({
    features: ['problem', 'level', 'type', 'solution', 'reasoning_formats', 'selected_modules', 'adapted_modules', 'reasoning_structure', 'reasoning', 'trajectory', 'answer_pred'],
    num_rows: 200
})

In [4]:
none_ds = ds.filter(lambda x: x["answer_pred"] == None)
none_ds

Filter:   0%|          | 0/200 [00:00<?, ? examples/s]

Dataset({
    features: ['problem', 'level', 'type', 'solution', 'reasoning_formats', 'selected_modules', 'adapted_modules', 'reasoning_structure', 'reasoning', 'trajectory', 'answer_pred'],
    num_rows: 2
})

In [5]:
import re
def map_fn(instance):
    if instance["answer_pred"] == None:
        text = "The final answer is:"
        pattern = fr"(?<={text}).*"
    
        response = instance["trajectory"]
    
        try:
            answer, trajectory = re.search(pattern, response).group(0).strip(), re.sub(pattern, "", response).replace(text, "").strip()
        except:
            answer, trajectory = None, response
    
        return {
            "trajectory": trajectory,
            "answer_pred": answer
        }

    return {
        "trajectory": instance["trajectory"],
        "answer_pred": instance["answer_pred"]
    }

new_ds = ds.map(map_fn)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [6]:
none_ds = new_ds.filter(lambda x: x["answer_pred"] == None)
none_ds

Filter:   0%|          | 0/200 [00:00<?, ? examples/s]

Dataset({
    features: ['problem', 'level', 'type', 'solution', 'reasoning_formats', 'selected_modules', 'adapted_modules', 'reasoning_structure', 'reasoning', 'trajectory', 'answer_pred'],
    num_rows: 0
})

In [69]:
index = 178
print(new_ds[index]["problem"])
print("+" * 80)
print(new_ds[index]["trajectory"])

The solutions of the equation $z^4+4z^3i-6z^2-4zi-i=0$ are the vertices of a convex polygon in the complex plane.  The area of this polygon can be expressed in the form $p^{a/b},$ where $a,$ $b,$ $p$ are positive integers, $p$ is prime, and $a$ and $b$ are relatively prime.  Find $a + b + p.$
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Here is the filled JSON for the given reasoning structure:

```
{
  "Problem Explanation": {
    "Identify the equation": "$z^4+4z^3i-6z^2-4zi-i=0$",
    "Explain the objective": "Find the area of the convex polygon formed by the solutions of the equation"
  },
  "Step 1: Factor the given equation": {
    "Method": "Use algebraic manipulations to factor the equation",
    "Expected outcome": "Factored form of the equation",
    "Outcome": "$z^4+4z^3i-6z^2-4zi-i = (z^2 + 2zi - i)(z^2 + 2zi + 1) = (z^2 + 2zi + 1)^2 - 2i(z^2 + 2zi + 1) + i^2$"
  },
  "Step 2: Identify the roots of the equation": {
    "Method": "Determin

In [7]:
new_ds.push_to_hub("sachithgunasekara/self-discover-mistral-original-MATH-eval")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/sachithgunasekara/self-discover-mistral-original-MATH-eval/commit/02ef67f3d149b43aac5d09743a23d60732d6cb63', commit_message='Upload dataset', commit_description='', oid='02ef67f3d149b43aac5d09743a23d60732d6cb63', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
from pylatexenc.latex2text import LatexNodes2Text

print(LatexNodes2Text().latex_to_text(ds[0]["problem"]))

Find all the rational roots of 2x^4 - x^3 - 18x^2 + 14x + 15 = 0.  Enter all the rational roots, separated by commas.


In [24]:
mdtex2html.convert(ds[1]["solution"])

'<p>We solve for <math xmlns="http://www.w3.org/1998/Math/MathML" display="inline"><mrow><msup><mi>x</mi><mn>2</mn></msup></mrow></math>: \\begin{align<em>}\n192x^2 -16 &amp;= 0\\\n192x^2 &amp;=16 \\\nx^2 &amp;= \\frac{1}{12}.\n\\end{align</em>} Therefore, <math xmlns="http://www.w3.org/1998/Math/MathML" display="inline"><mrow><msup><mi>x</mi><mn>4</mn></msup><mo>&#x0003D;</mo><mfrac><mrow><mn>1</mn></mrow><mrow><mn>144</mn></mrow></mfrac></mrow></math> and <math xmlns="http://www.w3.org/1998/Math/MathML" display="inline"><mrow><mfrac><mrow><mn>1</mn></mrow><mrow><msup><mi>x</mi><mn>4</mn></msup></mrow></mfrac><mo>&#x0003D;</mo><menclose notation="box"><mrow><mn>144</mn></mrow></menclose></mrow></math>.</p>'

In [8]:
from IPython.display import display, Latex
display(Latex(new_ds[178]["solution"]))

<IPython.core.display.Latex object>

In [28]:
from pylatexenc.latex2text import LatexNodes2Text

# Function to convert LaTeX to HTML
def latex_to_html(latex_text):
    return LatexNodes2Text().latex_to_text(latex_text)

html_solution = latex_to_html(ds[1]["solution"])
print(html_solution)

We solve for x^2: 
    192x^2 -16    = 0
    
    192x^2    =16 
    
    x^2    = 1/12.
 Therefore, x^4 = 1/144 and 1/x^4 = 144.


In [47]:
import json

def markdown_to_html(instance):
    return {
        "problem_html": markdown(instance["problem"]),
        "solution_html": markdown(instance["solution"]),
        "trajectory_html": f"<pre>{json.dumps(instance['trajectory'], indent=4)}</pre>"
    }

ds_html = ds.map(markdown_to_html)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [9]:
new_ds.to_csv("./henrycks_cm_original_evals.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

2375938

# Modified

In [3]:
par_dir = here("struct_vs_unstruct/data/mistral_large_2407/modified/non_self_synthesis/math/math-/math_eval")

In [4]:
ds = Dataset.load_from_disk(par_dir)
ds

Dataset({
    features: ['problem', 'level', 'type', 'solution', 'reasoning_formats', 'selected_modules', 'adapted_modules', 'reasoning_plan', 'reasoning', 'trajectory', 'answer_pred'],
    num_rows: 200
})

In [5]:
none_ds = ds.filter(lambda x: x["answer_pred"] == None)
none_ds

Filter:   0%|          | 0/200 [00:00<?, ? examples/s]

Dataset({
    features: ['problem', 'level', 'type', 'solution', 'reasoning_formats', 'selected_modules', 'adapted_modules', 'reasoning_plan', 'reasoning', 'trajectory', 'answer_pred'],
    num_rows: 2
})

In [6]:
import re
def map_fn(instance):
    if instance["answer_pred"] == None:
        text = "The final answer is:"
        pattern = fr"(?<={text}).*"
    
        response = instance["trajectory"]
    
        try:
            answer, trajectory = re.search(pattern, response).group(0).strip(), re.sub(pattern, "", response).replace(text, "").strip()
        except:
            answer, trajectory = None, response
    
        return {
            "trajectory": trajectory,
            "answer_pred": answer
        }

    return {
        "trajectory": instance["trajectory"],
        "answer_pred": instance["answer_pred"]
    }

new_ds = ds.map(map_fn)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [7]:
none_ds = new_ds.filter(lambda x: x["answer_pred"] == None)
none_ds

Filter:   0%|          | 0/200 [00:00<?, ? examples/s]

Dataset({
    features: ['problem', 'level', 'type', 'solution', 'reasoning_formats', 'selected_modules', 'adapted_modules', 'reasoning_plan', 'reasoning', 'trajectory', 'answer_pred'],
    num_rows: 2
})

In [10]:
print(none_ds[0]["trajectory"])

### Step-by-Step Reasoning Plan

1. **Identify the Core Mathematical Issue**
   - The core issue is to evaluate the expression \((r + s)(s + t)(t + r)\) using the roots \(r\), \(s\), and \(t\) of the polynomial \(x^3 + 9x^2 - 9x - 8\).

2. **Understand the Properties of the Roots**
   - Recognize that \(r\), \(s\), and \(t\) are the roots of the polynomial \(x^3 + 9x^2 - 9x - 8\).
   - Use Vieta's formulas to relate the coefficients of the polynomial to the sums and products of the roots.

3. **Apply Vieta's Formulas**
   - Vieta's formulas for a cubic polynomial \(ax^3 + bx^2 + cx + d = 0\) with roots \(r\), \(s\), and \(t\) are:
     - \(r + s + t = -\frac{b}{a}\)
     - \(rs + rt + st = \frac{c}{a}\)
     - \(rst = -\frac{d}{a}\)
   - For the polynomial \(x^3 + 9x^2 - 9x - 8\), the coefficients are \(a = 1\), \(b = 9\), \(c = -9\), and \(d = -8\).

4. **Calculate the Sums and Products of the Roots**
   - Using Vieta's formulas:
     - \(r + s + t = -9\)
     - \(rs + rt + st = -9\)


In [12]:
def map_fn(ins):
    if "First, let's expand \((r + s)(s + t)(t + r)\):" in ins["trajectory"]:
        return {
            "answer_pred": "\boxed{-486}"
        }

    return {
        "answer_pred": ins["answer_pred"]
    }

new_ds = new_ds.map(map_fn)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [14]:
none_ds = new_ds.filter(lambda x: x["answer_pred"] == None)
none_ds

Filter:   0%|          | 0/200 [00:00<?, ? examples/s]

Dataset({
    features: ['problem', 'level', 'type', 'solution', 'reasoning_formats', 'selected_modules', 'adapted_modules', 'reasoning_plan', 'reasoning', 'trajectory', 'answer_pred'],
    num_rows: 1
})

In [15]:
new_ds.push_to_hub("sachithgunasekara/self-discover-mistral-modified-MATH-eval")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/sachithgunasekara/self-discover-mistral-modified-MATH-eval/commit/00ea556a90a37635d332a703f8bdb576e5d418cf', commit_message='Upload dataset', commit_description='', oid='00ea556a90a37635d332a703f8bdb576e5d418cf', pr_url=None, pr_revision=None, pr_num=None)

In [16]:
new_ds.to_csv("./henrycks_cm_modified_evals.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1800573

In [10]:
file_name = "./project-5-at-2024-11-07-13-43-b93945eb.json"

In [11]:
with open(file_name, "r") as f:
    ds_ann = Dataset.from_list(json.load(f))

ds_ann

Dataset({
    features: ['problem', 'level', 'type', 'solution', 'reasoning_formats', 'selected_modules', 'adapted_modules', 'reasoning_structure', 'reasoning', 'trajectory', 'answer_pred', 'id', 'correct', 'annotator', 'annotation_id', 'created_at', 'updated_at', 'lead_time'],
    num_rows: 200
})

In [12]:
ds_ann[0]

{'problem': 'Find all the rational roots of $2x^4 - x^3 - 18x^2 + 14x + 15 = 0.$  Enter all the rational roots, separated by commas.',
 'level': 'Level 3',
 'type': 'Intermediate Algebra',
 'solution': 'By the Rational Root Theorem, the only possible rational roots are of the form $\\frac{a}{b},$ where $a \\mid 15$ and $b \\mid 2.$  Checking all possibilities, we find that the rational roots are $\\boxed{\\frac{5}{2},-3}.$',
 'reasoning_formats': '\n- should be the final answer based on calculations formatted in Latex style',
 'selected_modules': '1. How could I devise an experiment to help solve that problem?\n4. How can I simplify the problem so that it is easier to solve?\n9. How can I break down this problem into smaller, more manageable parts?\n10. Critical Thinking: This style involves analyzing the problem from different perspectives, questioning assumptions, and evaluating the evidence or information available. It focuses on logical reasoning, evidence-based decision-making, an

In [13]:
corr = ds_ann.filter(lambda x: x["correct"] == "True")
corr.num_rows / ds_ann.num_rows

Filter:   0%|          | 0/200 [00:00<?, ? examples/s]

0.675

In [14]:
ds_ann.push_to_hub("sachithgunasekara/self-discover-mistral-original-MATH-eval-annotated")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/sachithgunasekara/self-discover-mistral-original-MATH-eval-annotated/commit/c905ccc30b40f38095bb7dc273b8b14a4c5876b5', commit_message='Upload dataset', commit_description='', oid='c905ccc30b40f38095bb7dc273b8b14a4c5876b5', pr_url=None, pr_revision=None, pr_num=None)

# Modified Evals